# Example 2: Computation of Gradients and Non-adiabatic Couplings

In this tutorial it is shown, how to use the SA-OO-VQE solver in a simple step-by-step manner on a molecule of
formaldimine (methylene imine). It is shown, how to compute energies, gradients and non-adiabatic couplings
in several different settings w.r.t. a different number of optimized orbitals.

First of all, we'll specify geometry of the system.

In [20]:
geometry = [('N', [0.000000000000, 0.000000000000, 0.000000000000]),
            ('C', [0.000000000000, 0.000000000000, 1.498047000000]),
            ('H', [0.000000000000, -0.938765985000, 2.004775984000]),
            ('H', [0.000000000000, 0.938765985000, 2.004775984000]),
            ('H', [-0.744681452, -0.131307432, -0.634501434])]

Now we'll specify its properties, active space and the basis for Psi4 chemistry backend.

In [21]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

The next step is construction of `ProblemSet` instance - object containing all the information and necessary method for
our electronic structure problem.

In [22]:
import saoovqe

problem = saoovqe.problem.ProblemSet(geometry=geometry,
                                     charge=charge,
                                     multiplicity=multiplicity,
                                     n_electrons_active=n_elec_active,
                                     n_orbitals_active=n_orbs_active,
                                     basis_name=basis)

2023-06-23 10:13:30,397 - SAOOVQE.logger - INFO - SecondQuantizedProblem was created.


Now we need to create a set of circuits representing orthogonal states to construct the whole circuits representing
state vectors later.

In [23]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

2023-06-23 10:13:30,404 - SAOOVQE.logger - INFO - Circuits representing an orthogonal basis were created.


The next necessary part is to define an ansatz - it'll be also used to construct the state vector circuits later.

In [24]:
ansatz = saoovqe.Ansatz.from_ProblemSet(ansatz=saoovqe.AnsatzType.GUCCSD,
                                               problem=problem,
                                               repetitions=1,
                                               qubit_mapper=problem.fermionic_mapper)

2023-06-23 10:13:30,525 - SAOOVQE.logger - INFO - Ansatz was created.


And finally, now we can create an instance of our SA-OO-VQE solver. One of the main points is, orbital-optimization
can, but doesn't have to be used, or it can be used only on some molecular orbitals. We'll show all three cases here.
For no orbital-optimization it's enough to pass `None` to `orbital_optimization_settings` (it's also a default value).

In [25]:
from qiskit.primitives import Estimator

estimator = Estimator()

solver_no_oo = saoovqe.SAOOVQE(estimator=estimator,
                                                initial_circuits=initial_circuits,
                                                ansatz=ansatz,
                                                problem=problem,
                                                orbital_optimization_settings=None)

2023-06-23 10:13:30,533 - SAOOVQE.logger - INFO - SAOOVQE was created.


To specify number of optimized orbitals we can pass a dictionary to the parameter.

In [26]:
solver_oo_8 = saoovqe.SAOOVQE(estimator=estimator,
                                               initial_circuits=initial_circuits,
                                               ansatz=ansatz,
                                               problem=problem,
                                               orbital_optimization_settings={'n_mo_optim': 8})

2023-06-23 10:13:30,560 - SAOOVQE.logger - INFO - SAOOVQE was created.


And for all the orbitals to be optimized we can simply use its default behavior by passing an empty dictionary.

In [27]:
solver_oo_full = saoovqe.SAOOVQE(estimator=estimator,
                                                  initial_circuits=initial_circuits,
                                                  ansatz=ansatz,
                                                  problem=problem,
                                                  orbital_optimization_settings={})

2023-06-23 10:13:30,568 - SAOOVQE.logger - INFO - SAOOVQE was created.


Let's compare the energies now! To make the numerical optimizations, we'll use `SLSQP` optimizer provided by `SciPy`.
It may take a few minutes time now...

In [28]:
from qiskit.algorithms.optimizers import SciPyOptimizer
import numpy as np

optimizer = SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8})
energies_no_oo = solver_no_oo.get_energy(optimizer)
energies_oo_8 = solver_oo_8.get_energy(optimizer)
energies_oo_full = solver_oo_full.get_energy(optimizer)

print('\n============== State-Averaged Energies ==============')
print(np.mean(energies_no_oo))
print(np.mean(energies_oo_8))
print(np.mean(energies_oo_full))

2023-06-23 10:13:30,701 - SAOOVQE.logger - INFO - Computing energies...
2023-06-23 10:13:32,277 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.27262659  0.27262276 -0.0155197 ]
2023-06-23 10:13:32,548 - SAOOVQE.logger - INFO - Optimal phi angle for state-resolution was obtained (phi* = 3.806896263280463).
2023-06-23 10:13:33,174 - SAOOVQE.logger - INFO - Computing energies...
2023-06-23 10:13:34,748 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.27262659  0.27262276 -0.0155197 ]
2023-06-23 10:13:38,714 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.2518565   0.25185324 -0.01441605]
2023-06-23 10:13:42,822 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.2398443   0.23985171 -0.014324  ]
2023-06-23 10:13:47,151 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.23288984  0.23289705 -0.01427436]
2023-06-23 10:13:51,195 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.22875524  0.2287523  -0.0142461 ]
2023-0

-92.64256593087828
-92.64414483905284
-92.65981061428884


And now, let's have a look at the gradients of the potential energy surface for all the particles at both relevant
states.

In [29]:
print('\n============== Gradients ==============')
for state_idx in range(2):
    for atom_idx in range(len(geometry)):
        print(state_idx, atom_idx, solver_oo_full.eval_eng_gradient(state_idx, atom_idx))

2023-06-23 10:18:01,761 - SAOOVQE.logger - INFO - Constructing gradient evaluators...


0 0 [-0.22493245 -0.04894842 -0.20117543]
0 1 [0.07456918 0.01792904 0.23630368]
0 2 [-0.0030948   0.02465947 -0.02605988]
0 3 [-0.00630921 -0.02588515 -0.02038032]
0 4 [0.15976727 0.03224507 0.01131195]
1 0 [-0.04097219 -0.02459814 -0.54298526]
1 1 [0.06487503 0.00104745 0.51656241]
1 2 [-0.02686934  0.0313907  -0.02809013]
1 3 [-0.03876066 -0.03016646 -0.0273435 ]
1 4 [0.04172716 0.02232645 0.08185648]


And the non-adiabatic couplings.

In [30]:
print('\n============== Total non-adiabatic couplings ==============')
for atom_idx in range(len(geometry)):
    print(atom_idx, solver_oo_full.eval_nac(atom_idx))


============== Total non-adiabatic couplings ==============
0 [-0.55602127  2.35143966  1.28246372]
1 [-0.12923353 -1.09813364 -1.32051325]
2 [0.03576009 0.01765237 0.22183668]
3 [ 0.25309051  0.0851638  -0.10478533]
4 [ 0.36182224 -1.24872327 -0.07573726]


And finally, considering NACs, we can also have a look at CI and CSF NACs separately.

In [31]:
print('\n============== CI non-adiabatic couplings ==============')
for atom_idx in range(len(geometry)):
    print(atom_idx, solver_oo_full.ci_nacs[atom_idx])

print('\n============== CSF non-adiabatic couplings ==============')
for atom_idx in range(len(geometry)):
    print(atom_idx, solver_oo_full.csf_nacs[atom_idx])


============== CI non-adiabatic couplings ==============
0 [-0.51466843  2.13818324  1.28233387]
1 [-0.12248053 -1.06697028 -1.31961327]
2 [-0.01022644  0.01909822  0.22593759]
3 [ 0.29351684  0.08655607 -0.10886455]
4 [ 0.35385855 -1.17686726 -0.07979364]

============== CSF non-adiabatic couplings ==============
0 [-4.13528359e-02  2.13256419e-01  1.29842260e-04]
1 [-0.00675301 -0.03116336 -0.00089998]
2 [ 0.04598652 -0.00144585 -0.00410091]
3 [-0.04042633 -0.00139227  0.00407922]
4 [ 0.00796368 -0.07185602  0.00405638]
